In [ ]:
%pylab inline

This notebook contains code to reproduce the following from Mitra, et al.:

* Extended Data Fig. 4: Power analysis

In [ ]:
import os
import scipy.stats
OUTDIR="pdfs"

# Make editable in Illustrator
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# Contingency tables will have: status(0/1) x mutation(0/1)
# First, for different numbers of families, see how many mutations
# we need to see in probands (with 0 in controls) for genome-wide significance

NUMLOCI = 60000 # 900000 # approximate num genome-wide loci analyzed per family.
# should we use total number, or only total num with any mutations?
NUMMUTS = range(100)
NVALS = [2500] # sample size
power_vals = {} # N -> vals

for N in NVALS:
    # For each N, test different numbers of muts in proband
    pvals = []
    for aff_mut in NUMMUTS:
        table = [[aff_mut, 0], [N-aff_mut, N]]
        p = scipy.stats.fisher_exact(table)[1]
        pvals.append(p)
    power_vals[N] = pvals

fig = plt.figure()
ax = fig.add_subplot(111)
N = 2500 # Just plot for one sample size, since all look the same
ax.plot(NUMMUTS, [-1*np.log10(item) for item in power_vals[N]], label="Fisher p", color="black")
ax.axhline(y=-1*np.log10(0.05/NUMLOCI), color="gray", linestyle="dashed", label="Bonferonni threhsold p=0.05")
ax.set_xlabel("# Mut. in probands", size=15)
ax.set_ylabel("-log10P", size=15)
ax.set_xticklabels(ax.get_xticks(), size=12)
ax.set_yticklabels(ax.get_yticks(), size=12);
ax.legend()
fig.savefig(os.path.join(OUTDIR, "SuppFigure_Power.pdf"))

In [ ]:
# Figure: Add new plot: X-axis sample size, y-axis power. Different line for rate in cases.

SAMPSIZE = [1000, 5000, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 250000, 500000, 750000, 1000000]
PREV = [0.00001, 0.0001, 0.0005, 0.001, 0.003]
niter = 100

pthresh = 0.05/60000

fig = plt.figure()
ax = fig.add_subplot(111)

for pr in PREV:
    print("prev=%s"%pr)
    power = []
    for ssize in SAMPSIZE:
        pvals = []
        for i in range(niter):
            aff_mut = np.random.binomial(ssize, pr)
            table = [[aff_mut, 0], [ssize-aff_mut, ssize]]
            p = scipy.stats.fisher_exact(table)[1]
            pvals.append(p)
        power.append(np.mean([item<pthresh for item in pvals]))
    ax.plot(SAMPSIZE, power, label=pr)
plt.xscale("log")
ax.legend(loc="upper left")
fig.savefig(os.path.join(OUTDIR, "SuppFigure_Power_SampleSize.pdf"))